In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [68]:
path_2_train_file = 'train.csv'
df = pd.read_csv(path_2_train_file)

In [3]:
df.columns

Index(['label', 'id', 'visitTime', 'purchaseTime', 'hour', 'C1', 'C2', 'C3',
       'C4', 'C5', 'C6', 'C7', 'C8', 'N1', 'C9', 'N2', 'N3', 'N4', 'N5', 'N6',
       'N7', 'N8', 'N9', 'N10', 'C10', 'C11', 'C12'],
      dtype='object')

In [4]:
df['C1'].value_counts()

2569487005    8430
2108523568       7
2084406919       6
3438432991       5
256353298        5
              ... 
1427705163       1
4180196685       1
596547023        1
3559230936       1
77004800         1
Name: C1, Length: 22595, dtype: int64

In [5]:
df['C3'].value_counts()

1928325921    1232
3475347077    1212
2780744710    1179
1711803242     936
1822656032     536
              ... 
3540895230       1
2756181513       1
3550862535       1
1199484467       1
1533319149       1
Name: C3, Length: 2863, dtype: int64

In [6]:
df['C10'].value_counts()


3469607743    1425
3458698981      82
3183725626      73
1133005353      71
3662869197      68
              ... 
843115400        1
762569609        1
4142922634       1
830608273        1
487200771        1
Name: C10, Length: 28653, dtype: int64

In [7]:
label = df['label'].values


In [8]:
data = df.drop(columns=['label','id','visitTime','purchaseTime','C1','C3','C10']).values

In [9]:
X = data.copy()
y = np.array(label.copy())
y[:5]

array([-1, -1, -1, -1, -1], dtype=int64)

In [10]:
y = (y > 0)*1
nb_class = len(set(y))
print(nb_class)
id_Train = np.array([None]*nb_class)
id_Test = np.array([None]*nb_class)
for i in range(nb_class):
    id_i = np.where(y==i)[0]
    id_i_train, id_i_test = train_test_split(id_i, test_size=0.2)
    id_Train[i] = id_i_train
    id_Test[i] = id_i_test

id_Train = np.concatenate(id_Train)
id_Test = np.concatenate(id_Test)

X_train = X[id_Train]
Y_train = y[id_Train]
X_test = X[id_Test]
Y_test = y[id_Test]

2


In [11]:
X_train[0]

array([        18, 2541203883, 2308170622, 2967512603, 3153334760,
       2303407174, 3588031319,          0,  833114062,          0,
                0,          0,          0,          0,          0,
                0,          0,          0, 3617851689, 4220791646],
      dtype=int64)

In [12]:
id_toTrain = np.array([np.where(Y_train==i)[0] for i in range(nb_class)])

size_max = [len(id_toTrain[i]) for i in range(nb_class)]
print("before resampling")
print(size_max)

blc = 150
for i in range(len(size_max)):
    if size_max[i] > blc:
        size_max[i] = int(blc*(np.log10(size_max[i]/blc)+1))
    else:
        size_max[i] = int(blc/(np.log10(blc/size_max[i])+1))

print("after resampling")
print(size_max)
for i in range(nb_class):
    if len(id_toTrain[i]) > size_max[i]:
        id_toTrain[i], tmp = train_test_split(id_toTrain[i], test_size=1-size_max[i]/len(id_toTrain[i]))
    else:
        id_toTrain[i] = np.concatenate((id_toTrain[i], id_toTrain[i][np.random.randint(len(id_toTrain[i]), size=int(size_max[i]-len(id_toTrain[i])))]))
id_toTrain = np.concatenate(id_toTrain)
X_toTrain = X_train[id_toTrain]
Y_toTrain = Y_train[id_toTrain]

before resampling
[25096, 45]
after resampling
[483, 98]


In [41]:
len(X_toTrain)

581

In [42]:
len(Y_toTrain)

581

In [43]:
len(X_test)

6287

In [44]:
len(Y_test)

6287

In [60]:
# Fitting XGBoost to the Training set
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_toTrain, Y_toTrain)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [61]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)


In [66]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
cm

array([[6169,  106],
       [   2,   10]], dtype=int64)

In [63]:
print(classifier.score(X_test, Y_test))

0.9828216955622714


In [ ]:
# Testing on Test Set

In [95]:
path_2_test_file = 'test1.csv'
df_test1 = pd.read_csv(path_2_test_file)

In [96]:
df_test=df_test1.drop(columns=['label','id','visitTime','purchaseTime','C1','C3','C10'])
df_test.head()

,hour,C2,C4,C5,C6,C7,C8,N1,C9,N2,N3,N4,N5,N6,N7,N8,N9,N10,C11,C12
0,0,403381353,652481754,2967512603,580883881,2303407174,157533148,0,2088887026,0,0,0,0,0,0,0,0,0,3617851689,4220791646
1,0,1235983246,652481754,2116767313,571765677,2303407174,2374954832,0,2226682598,0,0,0,0,0,0,0,0,0,2379633508,1284343215
2,0,1235983246,652481754,2116767313,571765677,2303407174,2374954832,0,2226682598,0,0,0,0,0,0,0,0,0,3831440054,1213938795
3,0,403381353,652481754,2967512603,1607619193,1342367140,3588031319,4,3329757940,0,0,0,0,0,0,0,0,0,535829057,3828748840
4,0,2517724481,2308170622,2967512603,580883881,2303407174,157533148,0,2088887026,0,0,0,0,0,0,0,0,0,3617851689,4220791646


In [97]:
df_test.head()

,hour,C2,C4,C5,C6,C7,C8,N1,C9,N2,N3,N4,N5,N6,N7,N8,N9,N10,C11,C12
0,0,403381353,652481754,2967512603,580883881,2303407174,157533148,0,2088887026,0,0,0,0,0,0,0,0,0,3617851689,4220791646
1,0,1235983246,652481754,2116767313,571765677,2303407174,2374954832,0,2226682598,0,0,0,0,0,0,0,0,0,2379633508,1284343215
2,0,1235983246,652481754,2116767313,571765677,2303407174,2374954832,0,2226682598,0,0,0,0,0,0,0,0,0,3831440054,1213938795
3,0,403381353,652481754,2967512603,1607619193,1342367140,3588031319,4,3329757940,0,0,0,0,0,0,0,0,0,535829057,3828748840
4,0,2517724481,2308170622,2967512603,580883881,2303407174,157533148,0,2088887026,0,0,0,0,0,0,0,0,0,3617851689,4220791646


In [98]:
X_test[1]

array([         7,  403381353,  652481754, 2967512603, 3017120651,
       3649211661, 2206061889,          4, 2711086415,          0,
                0,          0,          0,          0,          0,
                0,          0,          0, 2379633508, 1284343215],
      dtype=int64)

In [99]:
a=df_test.to_numpy()

In [100]:
prediction = pd.DataFrame(classifier.predict_proba(a), columns=['Prob_0','Prob_1'])

In [101]:
prediction

,Prob_0,Prob_1
0,0.999532,0.000468
1,0.999853,0.000147
2,0.999872,0.000128
3,0.999326,0.000674
4,0.999230,0.000770
...,...,...
32081,0.987760,0.012240
32082,0.999624,0.000376
32083,0.999624,0.000376
32084,0.907855,0.092145


In [114]:
df_test1['Prob_0'] = prediction['Prob_0']
df_test1['Prob_1'] = prediction['Prob_1']


In [119]:
conditions=[(df_test1['Prob_0']> 0.5 ),
            (df_test1['Prob_0']<= 0.5 )]

In [129]:
choices=[-1,1]
df_test1['Pred']=np.select(conditions,choices,default='null')
df_test1['Pred']=pd.to_numeric(df_test1['Pred'])

In [130]:
df_test1

,label,id,visitTime,purchaseTime,hour,C1,C2,C3,C4,C5,...,N7,N8,N9,N10,C10,C11,C12,Prob_0,Prob_1,Pred
0,-1,1,19360733,-1,0,2569487005,403381353,681762206,652481754,2967512603,...,0,0,0,0,2128770044,3617851689,4220791646,0.999532,0.000468,-1
1,-1,2,19360734,-1,0,2416811818,1235983246,3653557526,652481754,2116767313,...,0,0,0,0,4256459078,2379633508,1284343215,0.999853,0.000147,-1
2,-1,3,19360735,-1,0,3765056181,1235983246,1310333812,652481754,2116767313,...,0,0,0,0,2840358457,3831440054,1213938795,0.999872,0.000128,-1
3,-1,4,19360737,-1,0,955802445,403381353,2290282995,652481754,2967512603,...,0,0,0,0,4039873369,535829057,3828748840,0.999326,0.000674,-1
4,-1,5,19360752,-1,0,2569487005,2517724481,2463986301,2308170622,2967512603,...,0,0,0,0,907610626,3617851689,4220791646,0.999230,0.000770,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32081,-1,32082,19446873,-1,23,2569487005,2018710401,887116886,652481754,2967512603,...,0,0,0,0,1546561973,3617851689,4220791646,0.987760,0.012240,-1
32082,-1,32083,19446914,-1,23,2569487005,403381353,3475347077,652481754,2967512603,...,0,0,0,0,3227742820,3617851689,4220791646,0.999624,0.000376,-1
32083,-1,32084,19446925,-1,23,2569487005,403381353,3475347077,652481754,2967512603,...,0,0,0,0,4069315994,3617851689,4220791646,0.999624,0.000376,-1
32084,-1,32085,19446931,-1,23,2569487005,2541203883,448895441,2308170622,2967512603,...,0,0,0,0,2724771230,3617851689,4220791646,0.907855,0.092145,-1


In [131]:
cm_test=confusion_matrix(df_test1['label'],df_test1['Pred'])

In [132]:
cm_test

array([[31728,   358],
       [    0,     0]], dtype=int64)

In [133]:
accuracy_score(df_test1['label'],df_test1['Pred'])

0.9888424858193605